# Retrieval Optimizer Comparison

In the example, we are going to conduct pairwise tests of 3 embedding models and 5 retrieval methods.

## Embedding models

- sentence-transformers/all-MiniLM-L6-v2
- sentence-transformers/all-mpnet-base-v2
- openai/text-embedding-3-small

## Search methods

- pure BM25
- vector search
- hybrid (BM25 + vector)
- rerank (cross-encoder/ms-marco-MiniLM-L-6-v2)
- weighted_rrf

## Installation

In [ ]:
%pip install redis-retrieval-optimizer openai

## Dataset

We'll import a dataset from the [beir benchmark IR project](https://github.com/beir-cellar/beir) to get going quickly. 

In [2]:
# Load data
from redis_retrieval_optimizer.corpus_processors import eval_beir

# check the link above for different datasets to try
beir_dataset_name = "nfcorpus"
data_folder = "data"

# Load sample data
corpus, queries, qrels = eval_beir.get_beir_dataset(beir_dataset_name)

09:50:55 beir.util INFO   Downloading nfcorpus.zip ...


./beir_datasets/nfcorpus.zip: 100%|██████████| 2.34M/2.34M [00:00<00:00, 4.07MiB/s]

09:50:56 beir.util INFO   Unzipping nfcorpus.zip ...
09:50:56 beir.datasets.data_loader INFO   Loading Corpus...



100%|██████████| 3633/3633 [00:00<00:00, 201821.23it/s]

09:50:57 beir.datasets.data_loader INFO   Loaded 3633 TEST Documents.
09:50:57 beir.datasets.data_loader INFO   Doc Example: {'text': 'Recent studies have suggested that statins, an established drug group in the prevention of cardiovascular mortality, could delay or prevent breast cancer recurrence but the effect on disease-specific mortality remains unclear. We evaluated risk of breast cancer death among statin users in a population-based cohort of breast cancer patients. The study cohort included all newly diagnosed breast cancer patients in Finland during 1995–2003 (31,236 cases), identified from the Finnish Cancer Registry. Information on statin use before and after the diagnosis was obtained from a national prescription database. We used the Cox proportional hazards regression method to estimate mortality among statin users with statin use as time-dependent variable. A total of 4,151 participants had used statins. During the median follow-up of 3.25 years after the diagnosis (rang

Now that we have our data we will save it locally to the gitignored `data/` folder

In [3]:
import os

os.makedirs(data_folder, exist_ok=True)

In [4]:
import json

with open(f"data/{beir_dataset_name}_corpus.json", "w") as f:
    json.dump(corpus, f)

with open(f"data/{beir_dataset_name}_queries.json", "w") as f:
    json.dump(queries, f)

with open(f"data/{beir_dataset_name}_qrels.json", "w") as f:
    json.dump(qrels, f)

# Execution

In [9]:
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key")

In [10]:
from redis_retrieval_optimizer.grid_study import run_grid_study
from redis_retrieval_optimizer.corpus_processors import eval_beir
from dotenv import load_dotenv

# load environment variables containing necessary credentials
load_dotenv()

redis_url = os.environ.get("REDIS_URL", "redis://localhost:6379/0")

metrics = run_grid_study(
    config_path="comparison_study_config.yaml",
    redis_url=redis_url,
    corpus_processor=eval_beir.process_corpus
)

09:53:12 redisvl.index.index INFO   Index already exists, overwriting.
09:53:13 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Recreating: loading corpus from file
09:53:13 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
09:53:14 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
09:53:14 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
09:53:15 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
09:53:16 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
09:53:16 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
09:53:16 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
09:53:17 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
09:53:17 httpx INFO   HTTP R

Batches: 100%|██████████| 1/1 [00:00<00:00, 39.76it/s]


Running search method: weighted_rrf
Recreating index with new embedding model
If using multiple embedding models assuming there is a json version of corpus available.
Recreating: loading corpus from file
09:58:09 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
09:58:09 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.94it/s]


09:58:23 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
09:58:23 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


...

Running search method: bm25
Running search method: vector


Batches: 100%|██████████| 1/1 [00:00<00:00, 31.06it/s]


Running search method: hybrid
Running search method: rerank
10:01:07 sentence_transformers.cross_encoder.CrossEncoder INFO   Use pytorch device: mps


Batches: 100%|██████████| 1/1 [00:00<00:00, 15.65it/s]


Running search method: weighted_rrf


In [11]:
metrics.to_csv("comparison_study_results.csv", index=False)

In [12]:
metrics[["search_method", "model", "model_dim", 'total_indexing_time', "avg_query_time", "recall", "precision", "ndcg"]].sort_values(by="ndcg", ascending=False)

,search_method,model,model_dim,total_indexing_time,avg_query_time,recall@k,precision,ndcg@k
1,vector,text-embedding-3-small,1536,0.60761,0.010809,0.183794,0.287616,0.242687
2,hybrid,text-embedding-3-small,1536,0.60761,0.005385,0.183794,0.287616,0.240194
4,weighted_rrf,text-embedding-3-small,1536,0.60761,0.007816,0.178586,0.270279,0.235438
14,weighted_rrf,sentence-transformers/all-mpnet-base-v2,768,0.66153,0.006898,0.173759,0.256656,0.224256
3,rerank,text-embedding-3-small,1536,0.60761,0.172249,0.172333,0.265015,0.221357
12,hybrid,sentence-transformers/all-mpnet-base-v2,768,0.66153,0.003456,0.166346,0.252632,0.214052
9,weighted_rrf,sentence-transformers/all-MiniLM-L6-v2,384,0.56115,0.003125,0.164964,0.244582,0.212325
11,vector,sentence-transformers/all-mpnet-base-v2,768,0.66153,0.006048,0.166461,0.252941,0.210091
13,rerank,sentence-transformers/all-mpnet-base-v2,768,0.66153,0.270323,0.175715,0.253870,0.209721
8,rerank,sentence-transformers/all-MiniLM-L6-v2,384,0.56115,0.184264,0.166997,0.253870,0.203366


In [13]:
metrics.columns

Index(['search_method', 'total_indexing_time', 'avg_query_time', 'recall@k',
       'ndcg@k', 'f1@k', 'precision', 'ret_k', 'algorithm', 'ef_construction',
       'ef_runtime', 'm', 'distance_metric', 'vector_data_type', 'model',
       'model_dim'],
      dtype='object')